# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [2]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')

# Получим общую информацию о датафрейме
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Видно что в некоторых колонках есть пропуски, с которыми будем разбираться на следующих шагах

In [3]:
# Посмотрим на первые 10 строк чтобы визуально оценить датафрейм
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Описание данных  
**children** — количество детей в семье  
**days_employed** — общий трудовой стаж в днях  
**dob_years** — возраст клиента в годах   
**education** — уровень образования клиента   
**education_id** — идентификатор уровня образования   
**family_status** — семейное положение   
**family_status_id** — идентификатор семейного положения   
**gender** — пол клиента   
**income_type** — тип занятости   
**debt** — имел ли задолженность по возврату кредитов   
**total_income** — ежемесячный доход   
**purpose** — цель получения кредита   

**Вывод**  
В колонке **days_employed** странные значения со знаком (-) и в float формате, а так же содежит пропуски  
В колонке **education** данные разного регистра  
В колонке **total_income** неизвестено в каких единицах выраженны данные, а так же содежит пропуски.  

Возможно потребуется сменить тип данных на int в **days_employed** и **total_income**

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
# Проверям количество пропусков и где они находятся
display(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# Посмотрим на первые 5 строк с пропусками и попытаемся обнаружить закономерности
display(data[data['days_employed'].isna()].head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [6]:
# Пропуски в колонках days_employed и total_income совпадают, возможно причина одна и та же
print(f"Количество пропущенных значений для days_employed: {data['days_employed'].isna().sum()}")

Количество пропущенных значений для days_employed: 2174


Пропуски нужно чем-то заменить, но чем?  
Колонка **total_income** должна содержать ежемесячный доход, но визуально это что-то другое. Возможно это доходы за 12 месяцев, но с форматом разберемся позже. Сначала заполним пропуски.  
В качестве решения хочу сгруппировать по колоке **income_type**, найти медиану на каждый тип занятости и потом заменить пропущенные значения медианой на данный тип занятости.

In [7]:
# Сгруппируем зарплату по типу занятости и посчитаем медиану на каждую группу.
display(data.groupby('income_type')['total_income'].median())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [8]:
# Добавим все уникальные значения типов занятости в отдельный лист
income_type_list = data['income_type'].unique()


#----------------------------------------------------------------------------------------
# # Создадим новый словарь который будет хранить типы занятости и медиану для них
# income_type_dict = {}

# # Найдем медиану для каждого типа и добавим в словарь income_type_dict
# for income_type in income_type_list:
#     median = data[data['income_type'] == income_type]['total_income'].median()
#     income_type_dict[income_type] = median

# # Создадим функцию которая получая тип занятости возвращает медиану    
# def income_type_fillna(income_type):
#     return income_type_dict[income_type]

# # Используя функцию income_type_fillna применим ее и заполним пропуски в total_income
# data['total_income'] = data['income_type'].apply(income_type_fillna)
#----------------------------------------------------------------------------------------

# Найдем медиану для каждого типа и заменим пропуски для каждого типа занятости
for income_type in income_type_list:
    median = data[data['income_type'] == income_type]['total_income'].median()
    data[data['income_type'] == income_type] = data[data['income_type'] == income_type].fillna(median)


# Проверим с новыми данными
display(data.groupby('income_type')['total_income'].median())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [9]:
print(f"Количество пропущенных значений для total_income: {data['total_income'].isna().sum()}")

Количество пропущенных значений для total_income: 0


Пропуски в колонке **total_income** заполнены, можно заняться пропусками в **days_employed**  

Как уже было замечено ранее, данные в колонке **days_employed** в странном формате float и даже есть с отрицательным значением, хотя должно быть целое число отображающее стаж в днях.  
Если маленькое значение стажа для человека в возрасте 50+ можно объяснить неофициальной трудовой занятостью, то отрицательные значения явно указывают на ошибку.  
В идеале конечно можно было бы запросить уточнение о причине возникновения таких данных, но т.к. для этого проекта эта колонка использоваться не будет, то просто заполним все данные медианой.

In [10]:
# Заменим пропуски медианным значением по колонке
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

**Вывод**

Файл содержит **21525** записей, часть из которых пропущена. Некоторые записи имеют различный регистр, а две колонки содержат неккоректный описанию формат и желательно, если есть возможность, уточнить у коллеги или клиента который предоставил файл, откорректировать данные до начала анализа.  
Часть данных в колонке **total_income** заменили медианой, несмотря на то что цифры не похожи на ежемесячный доход, воспримем эти данные как условные еденицы. Это не должно значительно отразиться на финальном результате.

### Замена типа данных

Т.к. в двух колонках содержатся странные данные не похожие на описание к файлу, заменим тип данных с float64 в int64 но результаты, для безопасности, сохраним в отдельные колонки

In [11]:
# Меняем тип данных на int64
data['days_employed_int'] = data['days_employed'].astype('int')
data['total_income_int'] = data['total_income'].astype('int')

In [12]:
# Т.к. новые колонки были добавлены в конце датафрейма, обновим порядок идексов
new_columns = ['children', 'days_employed', 'days_employed_int', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'total_income_int', 'purpose']

data = data[new_columns]

# Проверим если изменения применились
display(data.head(5))

,children,days_employed,days_employed_int,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_int,purpose
0,1,-8437.673028,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,253875,покупка жилья
1,1,-4024.803754,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,112080,приобретение автомобиля
2,0,-5623.422610,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,145885,покупка жилья
3,3,-4124.747207,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,267628,дополнительное образование
4,0,340266.072047,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,158616,сыграть свадьбу


**Вывод**

Заменил тип данных для двух колонках, а результат сохранил в новых. Чтобы на случай если старые данные вновь потребуются, они были доступны.  
Переместил колонки обновив порядок, в таком случае старые и новые данные будут находиться друг с другом.  
В таком новом формате будет удобнее работать еще и визуально. 

### Обработка дубликатов

При первом ознакомлении с файлом было заметно что в колонке **education** некоторые данные записаны в разном регистре.  
Изменим это прежде чем искать дубликаты.

In [13]:
# Переведем колонку с образованием в нижний регистр и проверим результат
data['education'] = data['education'].str.lower()
display(data.head(10))

,children,days_employed,days_employed_int,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_int,purpose
0,1,-8437.673028,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,253875,покупка жилья
1,1,-4024.803754,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,112080,приобретение автомобиля
2,0,-5623.422610,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,145885,покупка жилья
3,3,-4124.747207,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,267628,дополнительное образование
4,0,340266.072047,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,158616,сыграть свадьбу
5,0,-926.185831,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,255763,покупка жилья
6,0,-2879.202052,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,240525,операции с жильем
7,0,-152.779569,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,135823,образование
8,2,-6929.865299,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,95856,на проведение свадьбы
9,0,-2188.756445,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,144425,покупка жилья для семьи


In [14]:
# Посмотрим, сколько всего явных дубликатов найдено
print(data.duplicated().sum())

# А для наглядности посмотрим на количество записей для каждого типа занятости
display(data['income_type'].value_counts())

71


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [15]:
display(data[data.duplicated()])

,children,days_employed,days_employed_int,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_int,purpose
2849,0,142594.396847,142594,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,142594,покупка жилья для семьи
3290,0,118514.486412,118514,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,118514,сыграть свадьбу
4182,1,142594.396847,142594,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594.396847,142594,свадьба
4851,0,118514.486412,118514,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,118514,свадьба
5557,0,118514.486412,118514,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,118514.486412,118514,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,118514,дополнительное образование
21032,0,118514.486412,118514,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,118514,заняться образованием
21132,0,142594.396847,142594,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,142594,ремонт жилью
21281,1,142594.396847,142594,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,142594,покупка коммерческой недвижимости


In [16]:
# Отбросим все явные дубликаты
data = data.drop_duplicates().reset_index(drop=True)

# Посмотрим результат
print(data.duplicated().sum())

0


In [17]:
# Поиск неявных дубликатов в education
display(data['education'].unique())

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [18]:
# Поиск неявных дубликатов в family_status
display(data['family_status'].unique())

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Заметно что один из вариантов в **family_status** записан с заглавной буквы, исправим с помощью **.str.lower()**

In [19]:
# Приведем все к одному регистру и проверим
data['family_status'] = data['family_status'].str.lower()
display(data['family_status'].unique())

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [20]:
# Поиск неявных дубликатов в gender
display(data['gender'].unique())

array(['F', 'M', 'XNA'], dtype=object)

Один из вариантов записан как **XNA**, опечатка? Много ли таких записей?

In [21]:
# Проверим сколько всего таких записей
print(data[data['gender'] == 'XNA']['gender'].count())

1


Скорее всего опечатка и данных нет, возможно было указано **NA** (_not applicable_), заменим на **NA**

In [22]:
# Заменим на NA и проверим
data[data['gender'] == 'XNA'] = data.replace('XNA', 'NA')
display(data['gender'].unique())

array(['F', 'M', 'NA'], dtype=object)

In [23]:
# Поиск неявных дубликатов в income_type
display(data['income_type'].unique())

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [24]:
# Поиск неявных дубликатов в purpose
display(data['purpose'].unique())

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Скорее всего так будет долго разбираться и можно ошибиться, слишком много похожих слов.  
Попробуем вернуться к этому с помощью лемматизации ниже.

А пока проверим если появились новые дубликаты после форматирования некоторых колонок

In [25]:
# Проверим если появились новые дубликаты
print(data.duplicated().sum())
display(data['income_type'].value_counts())

0


сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

**Вывод**

Всего нашлось 71 дубликат, отбросив явные дубликаты, прошлись по каждой колонке и проверили на уникальные значения, разницу в написании и на возможные опечатки.  
Найденные недочеты были приведены к одному регистру, а опечатки исправлены.  
Обнаружили что есть много уникальных но похожих по смыслу и написанию фраз в колонке **purpose** но ее лучше обработать с помощью лемматизации.

### Лемматизация

In [26]:
# Имортируем pymystem3 для лемматизации
from pymystem3 import Mystem

# Counter использовался однажды, для составления словаря
# from collections import Counter

m = Mystem()

# Функция lemmatize_it для лемматизации возращает результат списком
def lemmatize_it(text):
    lemmas = m.lemmatize(text)
    return lemmas


#--------Блок ниже использовался для составления словаря,------------
#                 после чего был закомментирован
# -------------------------------------------------------------------
#
# Список уникальных фраз для лемматизации из столбца data['purpose']
#unique_purposes = data['purpose'].unique()
#
#
# Для каждой фразы найдем леммы и добавим их в словарь
# lemmatized_unique_purposes_dict = {}
#
# Используя функцию lemmatize_it и с подсчетом Counter записываем в словарь
# for unique_purpose in unique_purposes:
#     lemmas = lemmatize_it(unique_purpose)
#     lemmatized_unique_purposes_dict.update(Counter(lemmas))
# -------------------------------------------------------------------

    
# Словарь с отсортированными вручную категориями. 
sorted_purposes_dict = {
    'автомобиль': 'автомобиль',
    'образование': 'образование',
    'свадьба': 'свадьба',
    'недвижимость': 'недвижимость',
    'жилье': 'недвижимость',
}


In [27]:
# Функция purpose_category получает в качестве аргумента фразу, лемматизирует ее
# и после сверки со словарем возвращает нужную категорию.
def purpose_category(purpose):
    lemmatized_list = lemmatize_it(purpose)
    for lemma in lemmatized_list:
        if lemma in sorted_purposes_dict.keys():
            return sorted_purposes_dict[lemma]


#----------------------------- Дорогой ревьюер =) -----------------------------
#              я написал два варианта этой функции и тот что выше
#                  показался мне более элегантным, чем нижний.
#     Можешь подсказать и дать совет если есть более удобные и быстрые способы
#        пройти по колонке, лемматизировать все фразы, очистить от ненужного
# и сохранить в новую колонку кроме как сортировать вручную и в несколько шагов? 
#
#       Мне интересно на будущее и конечно если у тебя есть время на это.
#-------------------------------------------------------------------------------

# def ugly_purpose_category(purpose):
#     lemmatized_list = lemmatize_it(purpose)
#     if 'автомобиль' in lemmatized_list:
#         return 'автомобиль'
#     elif 'образование' in lemmatized_list:
#         return 'образование'
#     elif 'свадьба' in lemmatized_list:
#         return 'свадьба'
#     elif 'недвижимость' in lemmatized_list:
#         return 'недвижимость'
#     elif 'жилье' in lemmatized_list:
#         return 'недвижимость'
    
        
# Используя функцию purpose_category проверим каждую запись в data['purpose']
# а результат запишем в новую колонку data['lemmas_purpose']
data['lemmas_purpose'] = data['purpose'].apply(purpose_category)

# Проверим результат отобразив первые 10 записей
display(data.head(10))

,children,days_employed,days_employed_int,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_int,purpose,lemmas_purpose
0,1,-8437.673028,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,253875,покупка жилья,недвижимость
1,1,-4024.803754,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,145885,покупка жилья,недвижимость
3,3,-4124.747207,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,267628,дополнительное образование,образование
4,0,340266.072047,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,158616,сыграть свадьбу,свадьба
5,0,-926.185831,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,255763,покупка жилья,недвижимость
6,0,-2879.202052,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,240525,операции с жильем,недвижимость
7,0,-152.779569,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,135823,образование,образование
8,2,-6929.865299,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,95856,на проведение свадьбы,свадьба
9,0,-2188.756445,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,144425,покупка жилья для семьи,недвижимость


In [28]:
# Так же проверим если вдруг появилсиь пропущенные значения
data['lemmas_purpose'].isna().sum()

0

In [29]:
#----------Данный фрагмент кода использовался однажды для проверки функции purpose_category.----------
#                т.к. весь датафрейм довольно большой проверял на первых 50 строках,
#                            после успешной проверки код заккоментирован.
#
# df = data.head(50).copy()
# df['lemmas_purpose'] = df['purpose'].apply(purpose_category)
# df.head(10)
#-----------------------------------------------------------------------------------------------------

**Вывод**

С помощью лемматизации сначала взяли все уникальные значения из колонки **purpose** и сохранили в отдельный словарь, который вручную обработали удалив ненужное и объеденив одинаковое по смылсу для дальнейшего анализа.  
Некоторые функции и фрагменты кода использовались лишь раз, а после закомментированы.  
Используя полученный словарь и при помощью функции создали новую колонку и поместили туда ключевые слова, т.е., категории как результат лемматизации целей получения кредита.

### Категоризация данных

Подготавливаем данные чтобы ответить на вопросы о зависимости тех или иных факторов.  
Сгруппируем их в категории: 
- Дети, 
- Образование, 
- Семейное положение, 
- Доход, 
- Цели кредита

#### Группировка по наличию детей

In [29]:
# Проверим данные в колонке о наличии детей
display(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Некоторые данные выглядят неккоректными (-1 ребенок) и странными (20 детей)

In [30]:
# Заменим -1 на 1 в колонке с детьми
data[data['children'] == -1] = data.replace(-1, 1)
display(data['children'].unique())

array([ 1,  0,  3,  2,  4, 20,  5])

А значение 20 возможно было записано с опечаткой, лишним 0.  
Хоть и наличие у семьи 20 детей встречается в мире, это довольно редкое явление.

In [31]:
# Поэтому заменим 20 на 2
data[data['children'] == 20] = data.replace(20, 2)
display(data['children'].unique())


array([1, 0, 3, 2, 4, 5])

#### Группировка по образованию

In [32]:
# Проверим данные в колонке об образовании
display(data['education'].value_counts())

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

#### Группировка по семейному положению

In [33]:
# Проверим данные в колонке о семейном положении
display(data['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

#### Группировка по доходу

In [34]:
# Проверим данные в колонке о доходах
display(data['total_income_int'].value_counts())

142594    1070
172357     502
118514     387
150447     145
126262       3
          ... 
101387       1
138249       1
280240       1
390148       1
264193       1
Name: total_income_int, Length: 18608, dtype: int64

Выглядит внушительно, 18608 уникальных значений и это довольно большой разброс.  
Т.к. колонка **data['total_income']** с доходами имеет весьма странный вид для ежемесячного дохода и нет обяснения что это могло бы означать, к примеру значение в **158616.077870** для 53 летнего пенсионера.  
То в разделе **2.2 Замена типа данных** сменил формат колонки с float на int отбрсив значения после (.) и сохранив в новой колонке **data['total_income_int']**  
В таком случае останется 158616 и это целое число можно взять за условную единицу, т.н. **у.е.** с ней будет проще работать дальше.

Даже после форматирования все равно остается огромное количество различных значений, их лучше объеденить в группы.

In [35]:
# Проверим минимальное и максимальное значения
print(f"Минимальное значение в data['total_income_int']: {data['total_income_int'].min()}")
print(f"Максимальное значение в data['total_income_int']: {data['total_income_int'].max()}")

Минимальное значение в data['total_income_int']: 20667
Максимальное значение в data['total_income_int']: 2265604


In [36]:
# Напишем функцию сортировки по группам
def income_grouping(income):
    if income <= 500000:
        return '0 - 500 000 y.e.'
    if income <= 1000000:
        return '500 000 - 1 000 000 y.e.'
    if income <= 1500000:
        return '1 000 000 - 1 500 000 y.e.'
    if income <= 2000000:
        return '1 500 000 - 2 000 000 y.e.'
    if income <= 2500000:
        return '2 000 000 - 2 500 000 y.e.'
    return 'больше 2 500 000 y.e.'


# Создадим новую колонку и добавим туда результат функции income_grouping
data['income_group'] = data['total_income_int'].apply(income_grouping)

# Проверим что получилось
display(data.head(10))

,children,days_employed,days_employed_int,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,total_income_int,purpose,lemmas_purpose,income_group
0,1,-8437.673028,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,253875,покупка жилья,недвижимость,0 - 500 000 y.e.
1,1,-4024.803754,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,112080,приобретение автомобиля,автомобиль,0 - 500 000 y.e.
2,0,-5623.422610,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,145885,покупка жилья,недвижимость,0 - 500 000 y.e.
3,3,-4124.747207,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,267628,дополнительное образование,образование,0 - 500 000 y.e.
4,0,340266.072047,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,158616,сыграть свадьбу,свадьба,0 - 500 000 y.e.
5,0,-926.185831,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,255763,покупка жилья,недвижимость,0 - 500 000 y.e.
6,0,-2879.202052,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,240525,операции с жильем,недвижимость,0 - 500 000 y.e.
7,0,-152.779569,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,135823,образование,образование,0 - 500 000 y.e.
8,2,-6929.865299,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,95856,на проведение свадьбы,свадьба,0 - 500 000 y.e.
9,0,-2188.756445,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,144425,покупка жилья для семьи,недвижимость,0 - 500 000 y.e.


In [37]:
# Проверим данные в колонке с группами о доходах
display(data['income_group'].value_counts())

0 - 500 000 y.e.              21232
500 000 - 1 000 000 y.e.        197
1 000 000 - 1 500 000 y.e.       18
1 500 000 - 2 000 000 y.e.        5
2 000 000 - 2 500 000 y.e.        2
Name: income_group, dtype: int64

#### Группировка по целям на кредит

In [38]:
# Проверим данные в колонке о целях на кредит
display(data['lemmas_purpose'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemmas_purpose, dtype: int64

**Вывод**

Группировка некоторых данных помогла выявить неккоректные записи, такие как к пример -1 ребенок в семье, а огромнейший разброс в колонке с ежемесячным доходом сильно бы усложнил работу по анализу, если бы не сгруппировать суммы в условные категории.  
Все это позволяет оценить визуально с чем предстоит работать на следующем шаге, отвечая на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
display(data
        .groupby('children')
        .agg(families_with_children = ('children', 'count'),
             has_debt = ('debt', 'sum'),
             debt_share = ('debt', 'mean'))
        .sort_values(by='debt_share', ascending=False)
        .assign(percent = lambda data: data['debt_share'] * 100)
       )

,families_with_children,has_debt,debt_share,percent
children,,,,
4,41,4,0.097561,9.756098
2,2128,202,0.094925,9.492481
1,4855,445,0.091658,9.165808
3,330,27,0.081818,8.181818
0,14091,1063,0.075438,7.543822
5,9,0,0.000000,0.000000


**Вывод**

Судя по результатам таблицы, доля невозврата кредита самая высокая у семей с 4 детьми, но их всего 4 из 41.
А вот у семей с 1 ребенком, хоть доля невозврата кредита чуть ниже, но таких семей гораздо больше.  
Заемщиков без детей гораздо больше чем семей с детьми, а доля невозврата кредита ниже.  
Опираясь на полученные данные, можно сделать вывод что:  
Наличие детей повышает число невозврата по кредитам, чем больше детей тем выше число.

***

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
display(data
        .groupby('family_status')
        .agg(families = ('family_status', 'count'),
             has_debt = ('debt', 'sum'),
             debt_share = ('debt', 'mean'))
        .sort_values(by='debt_share', ascending=False)
        .assign(percent = lambda data: data['debt_share'] * 100)
       )

,families,has_debt,debt_share,percent
family_status,,,,
не женат / не замужем,2810,274,0.097509,9.750890
гражданский брак,4151,388,0.093471,9.347145
женат / замужем,12339,931,0.075452,7.545182
в разводе,1195,85,0.071130,7.112971
вдовец / вдова,959,63,0.065693,6.569343


**Вывод**

Из общего числа заемщиков, наибольший процент невозврата по кредитам приходится на тех кто **не женат / не замужем**  
А вот для тех кто **женат / замужем** этот показатель значительно ниже, возможно это связано с потенциальной поддержкой законного партнера, ведь даже для тех кто находится в **гражданском браке** доля невозврата близка к категории **женат / не замужем**  
У группы людей состоящих **в разводе** и **овдовевших** доля невозврата тоже низкая, возможно это связано с избеганием рисков с их стороны  
В итоге можно сделать вывод что оффициально женатые являются более надежными заемщиками.

***

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
display(data
        .groupby('income_group')
        .agg(income_group = ('income_group', 'count'),
             has_debt = ('debt', 'sum'),
             debt_share = ('debt', 'mean'))
        .sort_values(by='debt_share', ascending=False)
        .assign(percent = lambda data: data['debt_share'] * 100)
       )

,income_group,has_debt,debt_share,percent
income_group,,,,
2 000 000 - 2 500 000 y.e.,2,1,0.500000,50.000000
0 - 500 000 y.e.,21232,1727,0.081339,8.133949
500 000 - 1 000 000 y.e.,197,12,0.060914,6.091371
1 000 000 - 1 500 000 y.e.,18,1,0.055556,5.555556
1 500 000 - 2 000 000 y.e.,5,0,0.000000,0.000000


**Вывод**

На первый взгляд группа с доходом 2 000 000 - 2 500 000 y.e. имеет наибольший процент невозврата по кредитам в 50% но т.к. в этой группе всего 2 записи, делать такой вывод будет не корректно. 
В группе с доходом до 500 000 y.e. наибольшее число заемщиков и наивысший процент невозврата. При увелицении дохода, процент невозврата уменьшается.  
Можно прийти к выводу что чем выше уровень дохода, тем меньше риск невозврата.

***

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
display(data
        .groupby('lemmas_purpose')
        .agg(lemmas_purpose = ('lemmas_purpose', 'count'),
             has_debt = ('debt', 'sum'),
             debt_share = ('debt', 'mean'))
        .sort_values(by='debt_share', ascending=False)
        .assign(percent = lambda data: data['debt_share'] * 100)
       )

,lemmas_purpose,has_debt,debt_share,percent
lemmas_purpose,,,,
автомобиль,4306,403,0.093590,9.359034
образование,4013,370,0.092200,9.220035
свадьба,2324,186,0.080034,8.003442
недвижимость,10811,782,0.072334,7.233373


**Вывод**

Самой распространненой целью получения займа, получилась категория с недвижимостью. Это может быть покупка новой, ремонт и/или расширение существующей недвижимости как жилой так и коммерческой.  
Второй по частоте категорией является приобретение автомобиля, этой категории немного уступают кредиты на образование.

Не смотря на то что займы на недвижимость являются самыми востребованными, процент невозврата самый низкий в сравнении с другими из данного списка.  
Опираясь на это можно предположить и сделать вывод что на сделки с недвижимостью, если это жилая, решаются более обдумано, чем на сделки с автомобилями. Вероятно это может быть связано с тем что жилая недвижимость чаще является необходимостью, когда автомобиль, за исключением редких случаев, дополнительным "удовольствием".

## Шаг 4. Общий вывод

Влияют ли цели кредита на его возврат, думаю что да, но по одной только категории в любом случае нельзя принять решение о выдаче кредита. Ведь заемщик может быть официально женат / замужем, не иметь детей и относиться к группе с доходом 500 000 - 1 000 000 y.e. который планирует покупку автомобиля. 